# This notebook is used to determine the sensitivity matrix from CWFS data looking at both tip/tilt and decentering
## This is for data taken on 2021-06-08
### It first finds the pairs of CWFS images in the EFD
### It then fits the data a reports the zernikes
### Then plots the relationships and fits a slope for the matrix

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample, rendezvous_dataframes

%matplotlib inline

In [ ]:
#efd_client = EfdClient('summit_efd')
efd_client = EfdClient('ldf_stable_efd') 

### Find CWFS pairs
Query for all the `endReadout` events on the timespan of the night.

In [ ]:
# define axis you're interested in

In [ ]:
axis='y'
offset_start = Time("2021-06-09T04:44:00", format='isot', scale='tai') # time of where offsets to hexapod were applied

if axis == 'u':
    # times from when images were taken, 
    t1_set = Time("2021-06-09T05:01:00", format='isot', scale='tai')
    t2_set = Time("2021-06-09T05:21:44", format='isot', scale='tai')
if axis == 'v':
    t1_set = Time("2021-06-09T05:21:44", format='isot', scale='tai')
    t2_set = Time("2021-06-09T05:41:44", format='isot', scale='tai')
if axis == 'x':
    t1_set = Time("2021-06-09T05:41:44", format='isot', scale='tai')
    t2_set = Time("2021-06-09T06:04:44", format='isot', scale='tai')
if axis == 'y':
    t1_set = Time("2021-06-09T06:01:44", format='isot', scale='tai')
    t2_set = Time("2021-06-09T06:41:44", format='isot', scale='tai')

In [ ]:
end_readout = await efd_client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", 
                                           ["imageName", "requestedExposureTime", "additionalKeys",
                                            "additionalValues","timestampAcquisitionStart","timestampEndOfReadout"], t1_set, t2_set)

Now match each entry. For each `i` item with `intra` in the name, there must be an `i+1` with `extra` otherwise it is not a pair. 
A pair also has the same groupID
The image before the pair is an in-focus image.

In [ ]:
intra_images = []
extra_images = []
in_focus_images = []
intra_times = []
extra_times = []
in_focus_times = []
intra_exptimes = []
extra_exptimes = []
in_focus_exptimes = []

df=pd.DataFrame()

i = 0
npairs = 0
nmiss = 0

while i < len(end_readout)-2:
    intra = end_readout['imageName'][i]
    extra = end_readout['imageName'][i+1]
    in_focus = end_readout['imageName'][i-1]
    
    #skip known bad files
    if intra == 'AT_O_20200218_000179' and extra == 'AT_O_20200218_000180':
        i+=2
        continue
    # this is horrible, but looks for the CWFS pairs based on groupID
    # finds the in-focus image by seeing if an OBJECT was taken right before the pair
    # using a colon to separate values causes issues because there are colons in the timestamp!
    group_id=(end_readout.additionalValues[i])[0:25]
    group_id_next=(end_readout.additionalValues[i+1])[0:25]
    group_id_previous=(end_readout.additionalValues[i+2])[0:25] 
    image_type=(end_readout.additionalValues[i])[26::]
    image_type_previous=(end_readout.additionalValues[i-1])[26::]
    if ((group_id_next == group_id) and 
        (group_id_next != group_id_previous) and
        (image_type == 'ENGTEST' and image_type_previous == 'OBJECT')):
        
        print(f"Got a pair: {intra} x {extra}, with in-focus of {in_focus}")
        df_tmp=pd.DataFrame({'inFocus':end_readout['imageName'][i-1],
                             'intra':end_readout['imageName'][i],
                             'extra':end_readout['imageName'][i+1],
                             'inFocusExpTime':end_readout['requestedExposureTime'][i-1],
                             'inFocustimestampEndOfReadout':end_readout['timestampEndOfReadout'][i-1],
                             'intraExtratimestampAcquisitionStart':end_readout['timestampAcquisitionStart'][i],
                             'intraExtratimestampEndOfReadout':end_readout['timestampEndOfReadout'][i+1]},
                             index=[end_readout.index[i-1]])
        df=df.append(df_tmp)
        i+=2
        npairs+=1
    else:
#         print(f"No Match: {intra} x {extra}")
        nmiss+=1
        i+=1

print(f"Got {npairs} pairs and {nmiss} misses.")

In [ ]:
df

In [ ]:
# create new dataframe with new values of interest and we'll join them post-facto
df_offsets=pd.DataFrame()
# Populate the data structure from the pairs found above
for i in range(len(df.index)):
    
    # Determine time stamps for searching for metadata
    # include ability to correct for TAI if required, but set to zero for the moment

#    t1 = Time(in_focus_times[i], scale='tai') - TimeDelta(in_focus_exptimes[i], format='sec', scale='tai')
#    t2 = Time(extra_times[i], scale='tai') - TimeDelta(2., format='sec', scale='tai')
    
    # want time during CWFS sensing
    t1 = Time(df['intraExtratimestampAcquisitionStart'][i], format='unix_tai')
    t2 = Time(df['intraExtratimestampEndOfReadout'][i],format='unix_tai')
    


    azel = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                               ["elevationCalculatedAngle99", "azimuthCalculatedAngle99"], t1, t2)
    
    rotator = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                  ["nasmyth2CalculatedAngle99"], t1, t2)

    hexapod_vals = await efd_client.select_time_series("lsst.sal.ATHexapod.positionStatus", 
                                           ["reportedPosition0", "reportedPosition1", "reportedPosition2",
                                           "reportedPosition3", "reportedPosition4", "reportedPosition5"], t1 , t2)

    m1_pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure",
                                                  ["pressure"], t1, t2)
    

# For offsets we want to find the offsets between the start of the set and the beginning of the in-focus image, but the end works too
    cmd_offset = await efd_client.select_time_series("lsst.sal.ATAOS.command_offset",
                                                 ["u", "v", "w", "x", "y", "z"], offset_start , Time(df['inFocustimestampEndOfReadout'][i], format='unix_tai'))
    
    # to use the dataframe.between_time(), convert astropy Time object, to numpy time object, to pandas time object, and get the datetime.time
#     time1=pd.to_datetime(t1_set.to_datetime()).time()
#     time2=pd.to_datetime((Time(df['inFocustimestampEndOfReadout'][i],format='unix_tai')).to_datetime()).time()

    df_tmp=pd.DataFrame({'rot_pos':np.mean(rotator['nasmyth2CalculatedAngle99']),
                     'el':np.mean(azel['elevationCalculatedAngle99']),
                     'az':np.mean(azel['azimuthCalculatedAngle99']),
                     'x':hexapod_vals['reportedPosition0'].median(),
                     'y':hexapod_vals['reportedPosition1'].median(),
                     'z':hexapod_vals['reportedPosition2'].median(),
                     'u':hexapod_vals['reportedPosition3'].median(),
                     'v':hexapod_vals['reportedPosition4'].median(),
                     'w':hexapod_vals['reportedPosition5'].median(),
                     'm1': np.mean(m1_pressure['pressure']),
                     'hexXoffset': cmd_offset['x'].sum(),
                     'hexYoffset': cmd_offset['y'].sum(),
                     'hexUoffset': cmd_offset['u'].sum(),
                     'hexVoffset': cmd_offset['v'].sum(),
                     'hexZoffset': cmd_offset['z'].sum(),
                        },
                     index=[df.index[i]])
    df_offsets=df_offsets.append(df_tmp)

In [ ]:
df_offsets

In [ ]:
# Join the two dataframes to create a single one
df=df.join(df_offsets, lsuffix='_caller', rsuffix='_other')

In [ ]:
df

In [ ]:
# out = rendezvous_dataframes(end_readout, cmd_offset, direction='backward', tolerance=pd.Timedelta(days=1))

In [ ]:
filename="20210608_"+axis+"axis-motion_metadata.csv"

In [ ]:
df

In [ ]:
df.to_csv(filename)

# Now reduce the data for each pair to get the zernikes from fitting
#### You can start here if the top bit has already run 

In [ ]:
def atcs_get_bore_sight_angle(elevation_angle, nasmyth_angle):
    # modified from atcs.py
    # instrument on nasymth2, so
    parity_x = -1
    bore_sight_angle = elevation_angle + parity_x * nasmyth_angle + 90.0
    return bore_sight_angle

In [ ]:
import sys
import asyncio
import logging
import numpy as np
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
import time
import os

In [ ]:
# Read in the file (written using code above)
#df2 = pd.read_csv(filename)
df2 = pd.read_csv(filename, index_col=0)

In [ ]:
df2

In [ ]:
# Add all the zernike terms from the fitting
# this creates columns of NaNs and the loop below populates them
df2[['zern_defocus_nm', 'zern_xastig_nm','zern_yastig_nm',
    'zern_xcoma_nm', 'zern_ycoma_nm',
    'zern_xtrefoil_nm', 'zern_ytrefoil_nm','zern_spherical_nm' ]] = np.nan

In [ ]:
# this just needs to be set, but is not actually used
os.environ['LSST_DDS_DOMAIN'] = 'junk'

In [ ]:
# alignment script needs to have remotes set to False! Otherwise it'll try to command the hexapod!
script = LatissCWFSAlign(index=1, remotes=False)

In [ ]:
# define the location of the butler repo
script.dataPath='/project/shared/auxTel/'

In [ ]:
def get_visitID_from_filename(filename):
    # Expects AT_O_20200218_000167.fits
    # parse out visitID from filename - this is highly annoying
    tmp=filename.split('_')
    prefix=tmp[2] # dayobs without the dashes

    # Don't remember why I used int here... whitespace? 
    # surely fixable but bigger fish.
    suffix='{:05d}'.format(int(tmp[3].split('.')[0])) # SEQNUM, but need to trim extra 0 in obsid
    visitID = int((prefix+suffix))
    #print(visitID)
    return visitID

In [ ]:
# bin images when performing fits?
script.binning=2

In [ ]:
for n in range(len(df2)):
    # see tstn-015 and example notebook on running latiss_align_cwfs script
    script.intra_visit_id = get_visitID_from_filename(df2['intra'][n])
    script.extra_visit_id = get_visitID_from_filename(df2['extra'][n])
    script.angle = 90-atcs_get_bore_sight_angle(df2['el'][n], df2['rot_pos'][n])

    start_time=time.time()
    await script.run_cwfs()
    end_time=time.time()
    print('WFE fitting for visitIDs {0} and {1} took {2:0.3f} seconds'.format(script.intra_visit_id, script.extra_visit_id,end_time-start_time)) # 56.7s

    # Display fitting results?
    if (False):
        # plot zernikes
        x = np.arange(9)+4
        plt.plot(x, script.algo.zer4UpNm[:9], 'o-', label=f'{script.dz}')
        xlim = plt.xlim()
        plt.plot(np.arange(15), np.zeros(15)+50, 'b--')
        plt.plot(np.arange(15), np.zeros(15)-50, 'b--')
        plt.xlim(xlim)
        plt.ylabel("Zernike coeff (nm)")
        plt.xlabel("Zernike index")
        plt.grid()
        plt.legend()
        
    if (True):
        # plot image and mask
        fig1 = plt.figure(2, figsize=(12,8))
        ax11 = fig1.add_subplot(121)
        ax11.set_title("defocus 0.8 - intra")
        ax11.imshow(script.I1[0].image0)
        ax11.contour(script.algo.pMask) 
        ax12 = fig1.add_subplot(122)
        ax12.set_title("defocus 0.8 - extra")
        ax12.imshow(script.I2[0].image0)
        ax12.contour(script.algo.pMask) 
        
    # Put results into data structure 
    df2.loc[n,('zern_defocus_nm')] = script.algo.zer4UpNm[0]
    df2.loc[n,('zern_xastig_nm')] = script.algo.zer4UpNm[1]
    df2.loc[n,('zern_yastig_nm')] = script.algo.zer4UpNm[2]
    df2.loc[n,('zern_xcoma_nm')] = script.algo.zer4UpNm[3]
    df2.loc[n,('zern_ycoma_nm')]= script.algo.zer4UpNm[4]
    df2.loc[n,('zern_xtrefoil_nm')] = script.algo.zer4UpNm[5]
    df2.loc[n,('zern_ytrefoil_nm')] = script.algo.zer4UpNm[6]
    df2.loc[n,('zern_spherical_nm')] = script.algo.zer4UpNm[7]

In [ ]:
print('done')

In [ ]:
df2

In [ ]:
#write to CSV file
filename="20210607_"+axis+"axis_data_with_WFE_in_zerns.csv"
df2.to_csv(filename)

# Fit the data
### start here if just fitting existing data

In [ ]:
from scipy.optimize import curve_fit
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def parabola(x,b, x0, a):
    return b + a*(x-x0)**2 
def line(x,b, m):
    return b + m*x 
def invparabola(y,b,x0,a):
    return x0+np.sqrt((y-b)/a)
def invline(y,b,m):
    return (y-b)/m

### Set the axis you want to plot

In [ ]:
axis='y'
hexoffsetkey='hex'+axis.upper()+'offset'

In [ ]:
filename="20210607_"+axis+"axis_data_with_WFE_in_zerns.csv"
df3 = pd.read_csv(filename)

In [ ]:
df3

In [ ]:
# declare the indices that correspond to the axis of interest
if axis =='u':
    inds = np.arange(0,5)
if axis =='v':
    inds = np.arange(0,5)
if axis =='x':
    inds = np.arange(0,6)
if axis =='y':
    inds = np.arange(0,6)

In [ ]:
# using the variable axs for multiple Axes
nx=3; ny=3
fig, axs = plt.subplots(3, 3)
fig.set_size_inches(18, 12, forward=True)
fig.suptitle(f'WFE sensitivity from {hexoffsetkey}')
count=0
motion_key=hexoffsetkey
zern_keys = df3.keys()[24:35]
for i in range(nx):
    for j in range(ny):
        if count>len(zern_keys)-1:
            axs[i, j].annotate(f'Intentionally Blank',(0.3,0.5), xycoords=xycoords)
            continue
        zern_key=zern_keys[count]
        xdata=df3[motion_key][inds]
        ydata=df3[zern_key][inds]

        axs[i, j].plot(xdata,ydata,'o')
        x=np.arange(np.min(xdata), np.max(xdata), np.abs(np.max(xdata) - np.min(xdata))/100 )
        popt,pcov = curve_fit(line, xdata, ydata)
        perr = np.sqrt(np.diag(pcov))

        axs[i, j].plot(x,line(x, *popt))
        tmp=zern_key
        axs[i, j].set_ylabel(f"{zern_key.split('_')[1]} [nm]")
        axs[i, j].set_xlabel(motion_key+' [deg or mm]')
        xpos=0.6; ypos=0.8 ; xycoords='axes fraction'
        axs[i, j].annotate(f'm = {popt[1]:0.1f}\u00B1{perr[1]:0.1f}',(xpos,ypos), xycoords=xycoords)
        axs[i, j].annotate(f'b = {popt[0]:0.1f}\u00B1{perr[0]:0.1f}',(xpos,ypos-0.08), xycoords=xycoords)
        
        count+=1

In [ ]:
from lsst.ts.observing.utilities.auxtel.latiss.getters import get_image